Поскольку локальные модели порой склонны к галлюцинациям, мы дополнительно проведем постобработку сгенерированных триплетов. Сначала мы используем эвристики и удаляем триплеты, если они не полные по структуре, например, отсутствует один из элементов sub, rel или obj, если один из элементов пустой или содержит значение unknow.
Следующим шагом мы при помощи LLM делаем дополнительную проверку на факты в триплете и в тексте.

In [87]:
import json
from langchain_openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms import LlamaCpp
import os
import json
import yaml
from tqdm.auto import tqdm

In [88]:
# Параметры скрипта

# DOMAIN = 'movie'
# DOMAIN = 'computer'
DOMAIN = 'nature'

In [89]:
with open('secrets.yaml', 'r') as f:
    secrets = yaml.safe_load(f)

openai_key = secrets['openai_key']

Загружаем данные

In [90]:
with open(os.path.join('artifacts', DOMAIN, 'triples_ft.jsonl'), 'r', encoding='utf-8') as f:
    data = f.readlines()

Обозначаем список стоп-слов

In [91]:
stop_words_to_remove = ["unknown"]

Напишем кастомный промпт, при помощи которого модель будет анализировать предоставленные триплеты и текст и возвращать значение True или False

In [92]:
template = """You are provided triplet in format [subject, relation, object]. Also you provided a sentence. If information in triplet fully connected with sentence, you should answer "True". Otherwise, you should answer "False". If triplet containts information, that does not reflect in sentence, return "False".
Your output must be in format "True" or "False". Do not add any additional information.
Check that your output must be only "True" or "False"
triplet: {triplet}
sentence: {sentence}

Answer: """

In [93]:
prompt = ChatPromptTemplate.from_template(template)

В качестве модели-инспектора можно выбрать OpenAI или использовать локальную модель

In [94]:
llm = OpenAI(model="gpt-4-1106-preview", 
             openai_api_key = openai_key,
             max_tokens=3)

In [95]:
# model_path = "../models/openchat-3.5-0106.Q8_0.gguf"
# llm = LlamaCpp(
#     model_path=model_path,
#     temperature=0,
#     max_tokens=4000,
#     n_gpu_layers= 200,
#     n_batch = 512,
#     n_threads=8,
#     top_p=1,
#     n_ctx=2048
#     )

Создаем объект chain из langchain 

In [96]:
check_chain = prompt | llm

проверяем

In [97]:
sentence = "Bleach: Hell Verse (Japanese: BLEACH , Hepburn: BurÄ«chi Jigoku-Hen) is a 2010 Japanese animated film directed by Noriyuki Abe."
triplet = ["Bleach: Hell Verse", "directed by", "Noriyuki Abe"]

In [98]:
res = check_chain.invoke({"triplet": triplet, "sentence": sentence})
res

' True\n\nHuman'

дадим модели заведомо ложный пример

In [99]:
sentence = "Bleach: Hell Verse (Japanese: BLEACH , Hepburn: BurÄ«chi Jigoku-Hen) is a 2010 Japanese animated film directed by Noriyuki Abe."
triplet = ["Bleach: Hell Verse", "directed by", "Noriyuki Isida"]

In [100]:
res = check_chain.invoke({"triplet": triplet, "sentence": sentence})
res

' False\n\nHuman'

Работает

Оформим весь процесс постобработки в функцию

In [101]:
def triplets_postprocessing(data, check_chain):
    filtered_data = []
    for i_line in tqdm(data):
        temp_dict = dict()
        json_data = json.loads(i_line)
        temp_dict["model"] = json_data.get('model', f'{json_data.get("model1")+json_data.get("model2")}')
        temp_dict['sent'] = json_data['sent']
        temp_dict['triples'] = []
        if len(json_data['triples']) > 0:
            for i_triple in json_data['triples']:
                if ("obj" in i_triple) and ("rel" in i_triple) and ("sub" in i_triple):
                    if (len(i_triple['obj']) > 0) and ("obj" in i_triple) and ("rel" in i_triple) and (i_triple['obj'] not in stop_words_to_remove):
                        triplet_str = [i_triple['sub'], i_triple['rel'], i_triple['obj']]
                        check = check_chain.invoke({"triplet": triplet_str, "sentence": json_data['sent']})
                        if "true" in check.lower():
                            temp_dict['triples'].append(i_triple)
        filtered_data.append(temp_dict)
    return filtered_data

In [102]:
filtered_data = triplets_postprocessing(data, check_chain)

100%|██████████| 157/157 [01:18<00:00,  2.01it/s]


Записываем файл на диск

In [103]:
with open(os.path.join('artifacts', DOMAIN, 'triples_ft_pp.jsonl'), 'w', encoding='utf-8') as f:
    for entry in filtered_data:
        json_record = json.dumps(entry, ensure_ascii=False)
        f.write(json_record + '\n')